# COVID-19 A549

In [1]:
from clustergrammer2 import net
import pandas as pd

>> clustergrammer2 backend version 0.6.0


In [2]:
from copy import deepcopy

In [3]:
net.load_file('../data/gex_cat.tsv')
df_cat = net.export_df()
df_cat.head()

,"(Series1_NHBE_Mock_1, Cell Type: NHBE, Treatment: Mock)","(Series1_NHBE_Mock_2, Cell Type: NHBE, Treatment: Mock)","(Series1_NHBE_Mock_3, Cell Type: NHBE, Treatment: Mock)","(Series1_NHBE_SARS-CoV-2_1, Cell Type: NHBE, Treatment: SARS-CoV-2)","(Series1_NHBE_SARS-CoV-2_2, Cell Type: NHBE, Treatment: SARS-CoV-2)","(Series1_NHBE_SARS-CoV-2_3, Cell Type: NHBE, Treatment: SARS-CoV-2)","(Series2_A549_Mock_1, Cell Type: A549, Treatment: Mock)","(Series2_A549_Mock_2, Cell Type: A549, Treatment: Mock)","(Series2_A549_Mock_3, Cell Type: A549, Treatment: Mock)","(Series2_A549_SARS-CoV-2_1, Cell Type: A549, Treatment: SARS-CoV-2)",...,"(Series9_NHBE_IFNB_4h_1, Cell Type: NHBE, Treatment: IFNB)","(Series9_NHBE_IFNB_4h_2, Cell Type: NHBE, Treatment: IFNB)","(Series9_NHBE_IFNB_6h_1, Cell Type: NHBE, Treatment: IFNB)","(Series9_NHBE_IFNB_6h_2, Cell Type: NHBE, Treatment: IFNB)","(Series9_NHBE_IFNB_12h_1, Cell Type: NHBE, Treatment: IFNB)","(Series9_NHBE_IFNB_12h_2, Cell Type: NHBE, Treatment: IFNB)","(Series15_HealthyLungBiopsy_2, Cell Type: HealthyLungBiopsy, Treatment: 2)","(Series15_HealthyLungBiopsy_1, Cell Type: HealthyLungBiopsy, Treatment: 1)","(Series15_COVID19Lung_2, Cell Type: COVID19Lung, Treatment: 2)","(Series15_COVID19Lung_1, Cell Type: COVID19Lung, Treatment: 1)"
DDX11L1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
WASH7P,29,24,23,34,19,44,68,43,33,65,...,72,66,46,35,48,46,261,140,0,0
FAM138A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
FAM138F,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
OR4F5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df_cat.shape

(21797, 69)

In [5]:
cats = [x[1] for x in df_cat.columns.tolist()]
pd.Series(cats).value_counts()

Cell Type: A549                 29
Cell Type: NHBE                 24
Cell Type: A549-ACE2             6
Cell Type: Calu3                 6
Cell Type: COVID19Lung           2
Cell Type: HealthyLungBiopsy     2
dtype: int64

In [6]:
cats = [x[2] for x in df_cat.columns.tolist()]
pd.Series(cats).value_counts()

Treatment: Mock          26
Treatment: SARS-CoV-2    15
Treatment: IFNB           6
Treatment: IAV            6
Treatment: RSV            5
Treatment: IAVdNS1        4
Treatment: HPIV3          3
Treatment: 1              2
Treatment: 2              2
dtype: int64

### Load Diff Genes

In [7]:
df_diff = pd.read_excel('../data/primary_data/supp_table_2_diff_genes.xlsx', sheet_name='DESeq2-CoV (A549)', index_col=0)
df_diff.index.name = None
df_diff.head()

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,status
A1BG,76.406243,0.143778,0.273934,0.524865,0.599677,0.831619,OK
A1BG-AS1,52.452301,-0.174454,0.326569,-0.534203,0.593201,0.829007,OK
A1CF,5.316616,0.363328,0.924026,0.393201,0.694171,NaN,Low
A2M,0.000000,NaN,NaN,NaN,NaN,NaN,Low
A2M-AS1,24.292984,0.019341,0.453736,0.042625,0.966000,0.987279,OK


In [8]:
ser_qval = df_diff['padj']
ser_qval = ser_qval[ser_qval < 0.05]
ser_qval.shape
diff_genes = ser_qval.index.tolist()

In [9]:
df_cat.columns.tolist()[0]

('Series1_NHBE_Mock_1', 'Cell Type: NHBE', 'Treatment: Mock')

In [10]:
keep_cols = [x for x in df_cat.columns.tolist() if x[2] == 'Treatment: Mock' or x[2] == 'Treatment: SARS-CoV-2']
len(keep_cols)

41

In [11]:
df_comp = df_cat.loc[diff_genes, keep_cols]

/Users/nickfernandez/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


In [12]:
new_rows = [(x, 'log2-fold-change: ' + str(df_diff.loc[x, 'log2FoldChange'])) for x in df_comp.index.tolist()]

In [13]:
df_comp_cat = deepcopy(df_comp)
df_comp_cat.index = new_rows

In [14]:
net.load_df(df_comp_cat)
net.filter_cat(axis='col', cat_index=1, cat_name='Cell Type: A549')
net.filter_N_top(inst_rc='row', rank_type='var', N_top=1000)
net.normalize(axis='row', norm_type='zscore')
net.widget()

ExampleWidget(network='{"row_nodes": [{"name": "ADAR", "ini": 115, "clust": 76, "rank": 106, "rankvar": 42, "c…

In [15]:
!mkdir ../jsons/
net.save_dict_to_json(net.viz, '../jsons/A549_diff_genes.json')